In [1]:
import re
import spacy
import textract
import os
from spacy.matcher import Matcher
import glob
import os
import random
import resume_parser
import pandas as pd
import json

import stanza
import spacy_stanza
# stanza.download('en')
# import resumes_parser

In [2]:
resumes_dict = {
    'resume_id':
    {                  #filename for now but to be replaced with unique id
        'name': ('first name', 'last name'),
        'email': ('email', 'position_tuple'),
        'phone': ('phone', 'position_tuple'),
        'dob': ('dob', 'position_tuple'),
        'address': ('address', 'position_tuple'),
        'skills': [('skill1', 'position_tuple1'), ('skill1', 'position_tuple1')],
    }                     
}

In [3]:
# list of files to be processed
os.chdir("/home/anudeepadi/Documents/")
file_list = glob.glob(os.path.join(os.getcwd(), "Fw__Sample_resumes_", "*.docx"))
# extracting text from docx files
list_of_texts = [textract.process(file_path).decode() for file_path in file_list]

In [8]:
class parse:
    def __init__(self, resumes):
        self.resumes = resumes
        if type(resumes) == str:
            self.resumes = [resumes]
        self.nlp = spacy.load('en_core_web_md')
        self.matcher = Matcher(self.nlp.vocab)

    def extract_phone_number(self):
        ph_list = []
        for resume in self.resumes:
            items = re.finditer(r'\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}', resume)
            ph_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in items]
            ph_list.append(ph_span)
        return ph_list
    
    def extract_email(self):
        email_list = []
        for resume in self.resumes:
            items = re.finditer(r'[\w\.-]+@[\w\.-]+', resume)
            em_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in items]
            email_list.append(em_span)
        return email_list
       
    def extract_dob(self):
        dob_list = []
        for resume in self.resumes:
            result2 = re.finditer(r"[\d]{1,2}-[\d]{1,2}-[\d]{4}", resume) # DD-MM-YYYY
            dob_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in result2]
            dob_list.append(dob_span)
        return dob_list

    def extract_name(self):
        names_list = []
        for resume in self.resumes:
            doc = self.nlp(resume)
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    ns = list(ent.text)
            string = ''.join(ns)
            names_list.append(string)
        return names_list
        
    def extract_location(self):
        location_list = []
        for resume in self.resumes:
            doc = self.nlp(resume)
            for ent in doc.ents:
                if ent.label_ in ['GPE', 'LOC']:
                    ls = list(ent.text)
            string = ''.join(ls)
            location_list.append(string)
        return location_list
    
    def extract_skills(self):
        skills_list = []
        for resume in self.resumes:    
            nlp_text = self.nlp(resume)
            noun_chunks = nlp_text.noun_chunks
            # tokens = [token.text for token in nlp_text if not token.is_stop]
            tokens = [token.text for token in nlp_text if token.is_stop]
            
            data = pd.read_csv("/media/anudeepadi/New Volume/Coding --python/skills.csv") 
            skills = list(data.columns.values)

            skillset = []
            for token in tokens:
                if token.lower() in skills:
                    skillset.append(token)

            for token in noun_chunks:
                token = token.text.lower().strip()
                if token in skills:
                    skillset.append(token)

            st = [i.capitalize() for i in set([i.lower() for i in skillset])]
            skills_list.append(st)
        return skills_list

In [9]:
p = parse(list_of_texts)

In [10]:
p.extract_name()

['Jenkins',
 'Abinitio',
 'System Hardening',
 'Jenkins',
 'Srikanth Vadlamudi',
 'VIOS',
 'Medisys',
 'Salman Kaleem']

In [11]:
for resume in list_of_texts:
    names = p.extract_name()
    locations = p.extract_location()
    ph_numbers = p.extract_phone_number()
    emails = p.extract_email()
    dobs = p.extract_dob()
    skills = p.extract_skills()

In [ ]:
original_dict = {
    'AA_1_160322':{
    'name': names,
    'email': emails,
    'phone': ph_numbers,
    'dob': dobs,
    'location': locations,
    'skills': skills
    }
}

In [ ]:
original_dict

{'AA_1_160322': {'name': ['C',
   'L',
   'O',
   'U',
   'D',
   ' ',
   'A',
   'R',
   'C',
   'H',
   'I',
   'T',
   'E',
   'C',
   'T'],
  'email': [[('345@gmail.com', (24, 37))],
   [('12345@gmail.com', (24, 39))],
   [('122344@gmail.com', (12, 28))],
   [('abcd@gmail.com', (44, 58))],
   [('123451@gmail.com', (93, 109)),
    ('Srikanth50v51@gmail.com', (5294, 5317))],
   [('122690@gmail.com', (94, 110))],
   [('xxxxxi@gmail.com', (28, 44))],
   [('xyz@gmail.com', (46, 59)),
    ('eymemathew@gmail.com', (13099, 13119)),
    ('fyan@unr.edu', (13203, 13215)),
    ('SalmanKaleemM@Virtusa.com', (13282, 13307))]],
  'phone': [[('123-123-1234', (54, 66))],
   [('987-675-1234', (75, 87))],
   [('1234567', (54, 61)), ('997-1999', (3922, 3930))],
   [('(123) 456-5616', (62, 76))],
   [('1234567890', (59, 69))],
   [('123-466-7890', (116, 128))],
   [('123-456-7890', (15, 27)), ('707 205 6855', (46, 58))],
   [('123-456-7890', (25, 37))]],
  'dob': [[], [], [], [], [('03-10-1995', (5251,

In [ ]:
email_list = [email[0] for email in emails]
ph_list = [ph[0] for ph in ph_numbers]
names_list = [name for name in names]
loc_list = [loc for loc in locations]
skills_list = [skill for skill in skills]
dob_list = []
for dob in dobs:
    if dob:
        dob_list.append(dob[0])
    else:
        dob_list.append(('', (None, None)))

In [ ]:
zipped = zip(range(len(email_list)), names_list,email_list, ph_list, dob_list, loc_list, skills_list)

In [ ]:
original_dict = {}
for zip_ in zipped:
    original_dict[zip_[0]] = {}
    original_dict[zip_[0]]['name'] = zip_[1]
    original_dict[zip_[0]]['email'] = zip_[2]
    original_dict[zip_[0]]['phone'] = zip_[3]
    original_dict[zip_[0]]['dob'] = zip_[4]
    original_dict[zip_[0]]['location'] = zip_[5]
    original_dict[zip_[0]]['skills'] = zip_[6]

In [ ]:
original_dict

{0: {'name': 'C',
  'email': ('345@gmail.com', (24, 37)),
  'phone': ('123-123-1234', (54, 66)),
  'dob': ('', (None, None)),
  'location': 'India',
  'skills': ['Prototype',
   'Babel',
   'Js',
   'Cloud',
   'Mysql',
   'Sql server',
   'Apis',
   'Sdlc',
   'Xml',
   'Javascript',
   'Fulfillment',
   'Postgresql',
   'Json',
   'Scrum',
   'Flask',
   'Selenium',
   'Prototyping',
   'Windows',
   'Reports',
   'Oracle',
   'Test cases',
   'Banking',
   'Testing',
   'Workflows',
   'Process',
   'Sqlalchemy',
   'System',
   'Software development life cycle',
   'Analysis',
   'Pl/sql',
   'Transactions',
   'Design',
   'Java',
   'Html',
   'Jira',
   'Docker',
   'C',
   'Html5',
   'C++',
   'Jsp',
   'Css',
   'Servers',
   'Github',
   'Python',
   'Database',
   'Rest',
   'Sql',
   'Automation',
   'Soap',
   'Web services',
   'Mortgage',
   'Ui',
   'Aws']},
 1: {'name': 'L',
  'email': ('12345@gmail.com', (24, 39)),
  'phone': ('987-675-1234', (75, 87)),
  'dob': ('',

In [ ]:
with open('/media/anudeepadi/New Volume/Coding --python/extracted_data.json', 'w') as f:
    json.dump(original_dict, f)